In [68]:
#Libraries

#/usr/bin/python
from __future__ import print_function
import time 
import argparse
import pickle 
import numpy as np 
import os 
import math 
import random 
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import scipy.io

In [226]:
#Torch-bearer
import torch
from torch import nn, optim
from torch.nn import functional as F

from etm import ETM
from utils import *
from utils import nearest_neighbors, get_topic_coherence, get_topic_diversity

# 1. Data preprocessing 

In [70]:
abstracts = pd.read_csv("abstracts_eng.csv") #Replace with latest version
collection = list(abstracts['abstract'])
len(collection)

20494

#### Training in batches 
The idea is to simulate the real-time data stream.

In [115]:
seed = 11
random.seed(seed)
random.shuffle(collection)

print(len(collection))
streaming_batch_size = 20
batch1 = collection[:streaming_batch_size]
batch2 = collection[streaming_batch_size:2*streaming_batch_size]
batch3 = collection[2*streaming_batch_size:3*streaming_batch_size]

20494


# 2. The network components

## The embedding layer

---

In [72]:
# For the embeddings we'll use a pre-trained contextual model 
# Here we can play a bit to get a sense of its working 

#!pip install transformers 
from transformers import DistilBertTokenizerFast, DistilBertModel

Why DistilBer? <br>
The DistilBERT model was proposed in the blog post [Smaller, faster, cheaper, lighter: Introducing DistilBERT, a distilled version of BERT](https://medium.com/huggingface/distilbert-8cf3380435b5), and the paper DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter. DistilBERT is a small, fast, cheap and light Transformer model trained by distilling BERT base. It has 40% less parameters than bert-base-uncased, runs 60% faster while preserving over 95% of BERT’s performances as measured on the GLUE language understanding benchmark.

In [138]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased', return_dict=True)
inputs = tokenizer("Hello, we are three cool data scientists", return_tensors="pt")
outputs = model(**inputs)

In [196]:
outputs.last_hidden_state[0].data.cpu().numpy().shape

(10, 768)

---

Now there's a bit of work to do to prepare the embedding matrix. <br>
We want to use Bert to get the embedding for each word in our corpus. However, being Bert a contextual embedding we could end up with more than one embedding vector for each word. To solve this problem we associate each word a different token for each different embedding. 

In [ ]:
# I will work during the developing phase on a subset of the collection because 
# otherwise the memory requirements will be to high.
# We should later apply the same computation that I apply here to all the batches inside the 
# collection.
subset_size = 10 # len(collection)
collection_subset = collection[0:subset_size]

In [ ]:
tokenised_collection = tokenizer(collection_subset, return_tensors="pt", truncation=False, padding=True)
tokenised_collection["input_ids"][0] # we have added padding so that we can process the whole collection in a batch - we gain in speed

In [ ]:
embedded_collection = model(**tokenised_collection)

Now we can finally start to build our vocabulary. <br>
We'll look at each word and add it to the vocabulary only if there's not already the same embedding_vector in the collection.

In [ ]:
size = embedded_collection.last_hidden_state.size()
padding_size = size[1]
size

In [ ]:
idx2word = {} # vocabulary in the form (int,word) pairs
idx2bertIdx = {} # each index in our vocabulary is mapped to the Bert vocabulary index 
set_of_embeddings = set()


idx = 0 # initialise dictionary index 
new_token_ids = []

In [ ]:
# the idea of this for loop is the following: 

# input : [[4535,564,2342,...],[423423,32432,...],...] sequence of bert tokens for our collection 
#         [[embedding1, embedding2, ...],[...],...] and respective embedding vectors 


# output: [[0, 1, 2, 1, 4, ...], [0, 4, ...], ...] sequence of our tokens for our collection 
#         [[ embedding of token 0 ]
#          [ embedding of token 1 ]
#          [ embedding of token 2 ]
#          [ embedding of token 3 ]
#          [ embedding of token 4 ]
#                   
#                    ...
#
#          [ embedding of last token ]]

# Note: different tokens in our vocabulary (say 1 and 3) can refer to the same word if the embedding vector is different 

# The model will take as input the new tokens' collection (in some processed form maybe) and the embedding matrix
start = time.time()
for i in range(subset_size):
    t1 = time.time()
    embedded_doc = embedded_collection.last_hidden_state[i][tokenised_collection["attention_mask"][i].bool()] # removing padding using the attention mask 
    tokens_ids = tokenised_collection["input_ids"][i]
    new_token_ids_doc = []
    for j,emb_vector in enumerate(embedded_doc):
        token_id = tokens_ids[j].cpu().numpy() # bert current token 
        word = tokenizer.convert_ids_to_tokens([token_id])[0] # corresponding word
        bool_list = [torch.all(torch.eq(emb_vector, other)) for other in set_of_embeddings]
        if not any(bool_list): 
            # add new embedding to the set 
            set_of_embeddings.add(emb_vector)
            # increase the index and save the word in the dictionary 
            idx2word[idx] = word # save it in our vocabulary
            idx2bertIdx[idx] = token_id # save in id to bert id mapping
            new_token_ids_doc += [idx]
            idx += 1
        else: # find the right id for the word and add it to our new tokenisation
            word_id = list(idx2word.values()).index(word)
            new_token_ids_doc += [word_id]
    new_token_ids += [new_token_ids_doc]
    t2 = time.time()
    print("Document "+str(i)+" done. Time: "+str(round(t2-t1,2))+" s.")
end = time.time()
print("Total time: "+str(round(end-start,2))+" s.")

In [ ]:
len(set_of_embeddings) == len(idx2word.keys())

---

Now we write the loop that will process one batch in our collection. 

We also add 2 functionalities here: 
- **Cosine similarity clustering**. We add a filtering of the embedding vectors for each word based on cosine similarity, i.e. we only keep one of the vectors that are "*too close*" to each other. 
- **Stop-words removal**. We avoid to represent the stop-word in our embedding since they wouldn't anyway carry meaning for the topic and they would occupy a lot of memory since for [this](http://ai.stanford.edu/blog/contextual/) article they tend to be the most context dependent words in contextual embeddings.  

In [74]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # note: the words in this list are only lower case but distilbert tokenizer incorporates lower casing so we should be fine! read more here: https://huggingface.co/transformers/_modules/transformers/tokenization_distilbert_fast.html

In [75]:
def process_subset_cosine(doc_subset, tokenizer, model, set_of_embeddings, 
                          idx2word, new_token_ids, threshold=0.9):
    """ 
    Processing of a subset of the batch using cosine similarity clustering. 
    
    Parameters 
    ----- 
    doc_subset: list of documents (aka list of strings)
    tokenizer: instance of Bert tokenizer 
    model: instance of Bert model 
    set_of_embeddings: set containing the embedding vectors already in the vocabulary 
    idx2word: vocabulary mapping our token ids to the corresponding word. 
            Notice that each word can be mapped to multiple token ids.
    new_token_ids: representation of the collection with our token ids. 
    threshold: cosine similarity threshold. 
            Vectors with cosine similarity above the threshold are considered equal. 
            
    Returns 
    -----
    Updated versions of set_of_embeddings, idx2word and new_token_ids
    
    """
    
    tokenised_collection = tokenizer(doc_subset, return_tensors="pt", truncation=True, padding=True)
    print("tokenisation done")
    embedded_collection = model(**tokenised_collection)
    print("embeddings done")
    
    
    ##  preparing the variables we need ---------
    if len(idx2word) == 0:idx = 0
    else:idx = len(set_of_embeddings)
        
    cos = torch.nn.CosineSimilarity(dim = 0)
    
    subset_size = len(doc_subset)
    start = time.time()
    
    ## processing the collection document by document ----------
    for i in range(subset_size):
        t1 = time.time()
        embedded_doc = embedded_collection.last_hidden_state[i][tokenised_collection["attention_mask"][i].bool()] # removing padding using the attention mask 
        tokens_ids = tokenised_collection["input_ids"][i]
        new_token_ids_doc = []
        for j,emb_vector in enumerate(embedded_doc):
            
            token_id = tokens_ids[j].cpu().numpy() # bert current token 
            word = tokenizer.convert_ids_to_tokens([token_id])[0] # corresponding word
            # jump to the next token if the word is a stopword 
            if word in stop_words: continue 
            
            # cosine similarity clustering 
            bool_list = [cos(emb_vector, other) >= threshold for other in set_of_embeddings] 
            
            if (not any(bool_list)) or (word not in idx2word.values()): # we add the embedding anyway if we haven't encountered that word previously 
                # add new embedding to the set 
                set_of_embeddings.add(emb_vector)
                # increase the index and save the word in the dictionary 
                idx2word[idx] = word # save it in our vocabulary
                new_token_ids_doc += [idx]
                idx += 1
            else: # find the right id for the word and add it to our new tokenisation
                word_id = list(idx2word.values()).index(word)
                new_token_ids_doc += [word_id]
                
        new_token_ids += [new_token_ids_doc]
        t2 = time.time()
        if i%(subset_size//3)==0:print("Document "+str(i)+" done. Time: "+str(round(t2-t1,2))+" s.")
            
    end = time.time()
    print("Total time for the subset: "+str(round(end-start,2))+" s.")
    
    return set_of_embeddings, idx2word, new_token_ids

In [ ]:
# TESTING on a few example sentences 
idx2word = {} 
set_of_embeddings = set()
idx = 0 
new_token_ids = []

sen1 = "This mouse loves cheese"
sen2 = "I am using this mouse as a pointer"
doc = [sen1, sen2]
tokenised_collection = tokenizer(doc, return_tensors="pt", truncation=True, padding=True)
embedded_collection = model(**tokenised_collection)
set_of_embeddings, idx2word, new_token_ids = process_subset_cosine(doc, tokenizer, model, set_of_embeddings, idx2word, idx2bertIdx, new_token_ids)

In [76]:
def process_batch(batch, subset_size, tokeniser, model):
    """ Processing of a batch of documents in the collection. """
    
    ## initialisation 
    idx2word = {} 
    set_of_embeddings = set()
    idx = 0 
    new_token_ids = []
    subset_size = 25 
    
    
    start = time.time()

    ## processing the batch one subset at a time
    for s in range(0,batch_size,subset_size):
        print("Processing subset "+str(s//subset_size + 1))
        if s+subset_size < len(batch):batch_subset = batch[s:s+subset_size]
        else: batch_subset = batch[s:]
        set_of_embeddings, idx2word, new_token_ids = process_subset_cosine(batch_subset, tokenizer, model, 
                                                                           set_of_embeddings, idx2word, 
                                                                           new_token_ids, threshold=0.9)
        print("Number of word vectors so far: "+str(len(idx2word)))    
        print()
        
    end = time.time()
    print("Total time: "+str(round(end-start,2))+" s.")
    
    return set_of_embeddings, idx2word, new_token_ids

In [67]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [77]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased', return_dict=True)
set_of_embeddings, idx2word, new_token_ids = process_batch(batch2, 25, tokenizer, model)

Processing subset 1
tokenisation done
embeddings done
Document 0 done. Time: 0.7 s.
Document 6 done. Time: 26.51 s.
Document 12 done. Time: 47.25 s.
Document 18 done. Time: 64.05 s.
Total time for the subset: 596.8 s.
Number of word vectors so far: 2854

Total time: 613.63 s.


In [122]:
# build the embedding matrix from complete 'set_of_embeddings'
embedding = torch.stack(list(set_of_embeddings))
embedding.size()

torch.Size([2854, 768])

--- 
So now we finally have a vocabulary and an embedding matrix: everything is ready for our model! 

In [108]:
vocab_size = len(idx2word)
vocab_size

2854

In [ ]:
# adjust the get_batch function so as to get the documents x vocabulary matrix 
# required by the model starting from the complete 'new_token_ids' list

In [80]:
def get_batch(corpus, ind, vocab_size, device, emsize=300):
    """
    This function takes as input a list of tokenised documents (corpus)
    and the indices of the documents in the batch (ind)
    and returns as output the torch tensor to feed into the net. 
    The list of documents defines the batch to work on. 
    """
    batch_size = len(ind)
    data_batch = np.zeros((batch_size, vocab_size))
    
    for i, doc_id in enumerate(ind):
        doc = corpus[doc_id]
        L = len(doc)
        if doc_id != -1:
            for word_id in doc:
                counts = doc.count(word_id)
                data_batch[i, word_id] = counts
    data_batch = torch.from_numpy(data_batch).float().to(device)
    return data_batch

## The model!

Hyperparameters

In [131]:
### model-related arguments
num_topics = 5
rho_size = 768 # dimension of rho 
emb_size = 768 # dimension of embeddings 
t_hidden_size = 600 # dimension of hidden space of q(theta)
theta_act = 'relu' # either tanh, softplus, relu, rrelu, leakyrelu, elu, selu, glu
train_embeddings = False
seed = 11

### optimization-related arguments
lr = 0.005
lr_factor = 4.0 #divide learning rate by this
epochs = 20 
mode = "train"
enc_drop = 0.0 # dropout rate on encoder
clip = 0.0 # gradient clipping
nonmono = 10 # number of bad hits allowed ...?
weight_decay = 1.2e-6
anneal_lr = False # whether to anneal the learning rate or not
bow_norm = True # normalize the bows or not 
_optimizer = "adam"

### evaluation, visualization, and logging-related arguments
num_docs_train = 18
num_words = 10 #number of words for topic viz'
log_interval = 2 #when to log training
visualize_every = 10 #when to visualize results
tc = False # whether to compute topic coherence or not
td = False # whether to compute topic diversity or not

### data and file related arguments
save_path = './results'
batch_size = 20
eval_batch_size = 20 #input batch size for evaluation
load_from = "" #the name of the ckpt to run evaluation from


Random seed 

In [88]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Setting the random seed 
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

Checkpoints

In [90]:
if not os.path.exists(save_path):
    os.makedirs(save_path)
if mode == 'eval':
    ckpt = load_from 
else:
    ckpt = os.path.join(save_path, 
        'etm_K_{}_Htheta_{}_Optim_{}_Clip_{}_ThetaAct_{}_Lr_{}_Bsz_{}_RhoSize_{}_trainEmbeddings_{}'.format(
         num_topics, t_hidden_size, _optimizer, clip, theta_act, lr, batch_size, rho_size, train_embeddings))


Optimizer 

In [93]:
if _optimizer == 'adam':
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=lr_factor)
elif _optimizer == 'adagrad':
    optimizer = optim.Adagrad(model.parameters(), lr=lr, weight_decay=lr_factor)
elif _optimizer == 'adadelta':
    optimizer = optim.Adadelta(model.parameters(), lr=lr, weight_decay=lr_factor)
elif _optimizer == 'rmsprop':
    optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=lr_factor)
elif _optimizer == 'asgd':
    optimizer = optim.ASGD(model.parameters(), lr=lr, t0=0, lambd=0., weight_decay=lr_factor)
else:
    print('Defaulting to vanilla SGD')
    optimizer = optim.SGD(model.parameters(), lr=args.lr)

Finally the model 

In [281]:
# define model
etm_model = ETM(num_topics = num_topics, 
            vocab_size = vocab_size, 
            t_hidden_size = t_hidden_size, 
            rho_size = rho_size, 
            emsize = emb_size, 
            theta_act = theta_act, 
            embeddings = embedding,
            train_embeddings = train_embeddings, 
            enc_drop = enc_drop).to(device)

print('model: {}'.format(etm_model))


model: ETM(
  (t_drop): Dropout(p=0.0, inplace=False)
  (theta_act): ReLU()
  (alphas): Linear(in_features=768, out_features=5, bias=False)
  (q_theta): Sequential(
    (0): Linear(in_features=2854, out_features=600, bias=True)
    (1): ReLU()
    (2): Linear(in_features=600, out_features=600, bias=True)
    (3): ReLU()
  )
  (mu_q_theta): Linear(in_features=600, out_features=5, bias=True)
  (logsigma_q_theta): Linear(in_features=600, out_features=5, bias=True)
)


# 3. Training

In [282]:
train_corpus = new_token_ids[:num_docs_train]
test_corpus = new_token_ids[num_docs_train:]

In [283]:
def train(model, epoch, corpus, num_docs_train=num_docs_train, batch_size=batch_size, vocab_size=vocab_size, 
          bow_norm=bow_norm, clip=clip, log_interval=log_interval):
    """ Just the training function ... """
    
    model.train() #setting the model in training mode
    # preparing all the data structures 
    acc_loss = 0
    acc_kl_theta_loss = 0
    cnt = 0
    indices = torch.randperm(num_docs_train)
    indices = torch.split(indices, batch_size)
    
    for idx, ind in enumerate(indices): # all our batches 
        optimizer.zero_grad()
        model.zero_grad()
        data_batch = get_batch(corpus, ind, vocab_size, device)
        sums = data_batch.sum(1).unsqueeze(1) # what are we summing ?? 
        
        # maybe normalising the input 
        if bow_norm: normalized_data_batch = data_batch / sums
        else: normalized_data_batch = data_batch
        # loss on the batch 
        recon_loss, kld_theta = model(data_batch, normalized_data_batch)
        total_loss = recon_loss + kld_theta
        total_loss.backward() # compute backpropagation
        # maybe clip the gradient 
        if clip > 0: torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step() # finally update the weights 
        # accumulate the total loss 
        acc_loss += torch.sum(recon_loss).item()
        acc_kl_theta_loss += torch.sum(kld_theta).item()
        cnt += 1
        
        # visualisation/print time! ('cur' stands for current ...)
        if idx % log_interval == 0 and idx > 0:
            cur_loss = round(acc_loss / cnt, 2) 
            cur_kl_theta = round(acc_kl_theta_loss / cnt, 2) 
            cur_real_loss = round(cur_loss + cur_kl_theta, 2)
            print('Epoch: {} .. batch: {}/{} .. LR: {} .. KL_theta: {} .. Rec_loss: {} .. NELBO: {}'.format(
                epoch, idx, len(indices), optimizer.param_groups[0]['lr'], cur_kl_theta, cur_loss, cur_real_loss))
    
    # Wrapping up the results of the epoch! 
    cur_loss = round(acc_loss / cnt, 2) 
    cur_kl_theta = round(acc_kl_theta_loss / cnt, 2) 
    cur_real_loss = round(cur_loss + cur_kl_theta, 2)
    print('*'*100)
    print('Epoch----->{} .. LR: {} .. KL_theta: {} .. Rec_loss: {} .. NELBO: {}'.format(
            epoch, optimizer.param_groups[0]['lr'], cur_kl_theta, cur_loss, cur_real_loss))
    print('*'*100)

In [284]:
def visualize(m, num_topics=num_topics, num_words=num_words, 
              vocab=idx2word, show_emb=True, tokenizer=tokenizer, bert_model=model):
    """ This is a cool visualisation function. 
    Takes as input the model so far and shows the discovered embeddings! """
    
    # We're going to save our results here 
    # TODO: parametrize this path 
    if not os.path.exists('./results'):
        os.makedirs('./results')

    m.eval() #set the net in evaluation mode 
    # set a few words to query 
    queries = ['andrew', 'computer', 'sports', 'religion', 'man', 'love', 
                'intelligence', 'money', 'politics', 'health', 'people', 'family']

    ## visualize topics using monte carlo (sampling from the posterior I guess)
    with torch.no_grad(): # no gradients computation - makes forward pass lighter 
        print('#'*100)
        print('Visualize topics...')
        topics_words = []
        gammas = m.get_beta() # topics distributions 
        for k in range(num_topics):
            gamma = gammas[k]
            top_words = list(gamma.cpu().numpy().argsort()[-num_words+1:][::-1])
            topic_words = [vocab[a] for a in top_words] 
            topics_words.append(' '.join(topic_words))
            print('Topic {}: {}'.format(k, topic_words))

        if show_emb:
            ## visualize word embeddings by using V to get nearest neighbors
            print('#'*100)
            print('Visualize word embeddings by using output embedding matrix')
            
            # extract the embeddings from the model! 
            try:embeddings = m.rho.weight  # Vocab_size x E
            except:embeddings = m.rho         # Vocab_size x E
            
            
            for word in queries:
                # extracting Bert representation of the word
                inputs = tokenizer(word, return_tensors="pt")
                outputs = bert_model(**inputs).last_hidden_state[0]
                if outputs.size()[0]>1: #aggregate
                    outputs = torch.sum(outputs, dim=0)
                nns = utils.nearest_neighbors(q=outputs, 
                                        embeddings=embeddings, vocab=list(vocab.values()))
                print('word: {} .. neighbors: {}'.format(word, nns)) # utility function 
            print('#'*100)

In [285]:
def evaluate(m, corpus, num_docs_test, tc=tc, td=td, 
             eval_batch_size=eval_batch_size, vocab_size=vocab_size, 
             bow_norm=bow_norm):
    """
    Evaluating the trained model on the test set using either perplexity, or coherence and diversity. 
    Compute perplexity on document completion.
    """
    
    m.eval() # set model in evaluation mode 
    with torch.no_grad():
        indices = torch.split(torch.tensor(range(num_docs_test)), eval_batch_size)
        
        ## get \beta here
        beta = m.get_beta()

        ### do dc and tc here
        acc_loss = 0
        cnt = 0
        
        for idx, ind in enumerate(indices):
            data_batch = get_batch(corpus, ind, vocab_size, device)
            sums = data_batch.sum(1).unsqueeze(1)
            if bow_norm: normalized_data_batch = data_batch / sums
            else: normalized_data_batch = data_batch
                
            ## get theta
            theta, _ = m.get_theta(normalized_data_batch)
            ## get prediction loss
            res = torch.mm(theta, beta)
            preds = torch.log(res)
            recon_loss = -(preds * data_batch).sum(1)
            loss = recon_loss / sums.squeeze()
            loss = loss.mean().item()
            acc_loss += loss
            cnt += 1
        
        # Calculate final loss 
        cur_loss = acc_loss / cnt
        ppl_dc = round(math.exp(cur_loss), 1)
        print('*'*100)
        print('{} Doc Completion PPL: {}'.format(source.upper(), ppl_dc))
        print('*'*100)
        
        
        if tc or td: # calculate topic coherence or topic diversity 
            beta = beta.data.cpu().numpy()
            if tc:
                print('Computing topic coherence...')
                get_topic_coherence(beta, train_tokens, vocab)
            if td:
                print('Computing topic diversity...')
                get_topic_diversity(beta, 25)
        return ppl_dc

Here is the code that actually launches the training 

In [286]:
import importlib
utils = importlib.reload(utils)

In [287]:
# Initialising the data structures 
best_epoch = 0
best_val_ppl = 1e9
all_val_ppls = []

# Let's get a sense of how bad the model is before training 
print('\n')
print('Visualizing model quality before training...')
visualize(etm_model)
print('\n')


for epoch in range(1, epochs):
    
    train(etm_model, epoch, train_corpus) # train 
    num_docs_test = len(new_token_ids) - num_docs_train
    val_ppl = evaluate(etm_model, test_corpus, num_docs_test) # evaluate 
    
    # only saving the model if it's the best so far 
    if val_ppl < best_val_ppl: 
        with open(ckpt, 'wb') as f:
            torch.save(model, f)
        best_epoch = epoch 
        best_val_ppl = val_ppl
        
    else:
        ## check whether to anneal lr (aka decreasing it by a constant factor )
        lr = optimizer.param_groups[0]['lr']
        if args.anneal_lr and (len(all_val_ppls) > args.nonmono and val_ppl > min(all_val_ppls[:-args.nonmono]) and lr > 1e-5):
            optimizer.param_groups[0]['lr'] /= args.lr_factor
            
    #maybe visualise 
    if epoch % args.visualize_every == 0:
        visualize(etm_model)
        
    #save perplexities 
    all_val_ppls.append(val_ppl)



Visualizing model quality before training...
####################################################################################################
Visualize topics...
Topic 0: ['community', 'production', 'around', '##le', 'prices', '-', 'arrest', 'american', 'rates']
Topic 1: [')', 'bottom', 'conditions', '##irs', 'hypothesis', 'elbow', 'due', '##tify', '##ts']
Topic 2: ['function', 'date', '(', '##ening', '##yn', 'performance', 'inside', '##c', 'com']
Topic 3: ['stress', '(', 'structures', '’', ',', '##ors', 'aged', 'data', '##cl']
Topic 4: ['##s', '##uti', 'retain', 'greater', 'new', 'convergence', 'indicate', '##fp', 'compared']
####################################################################################################
Visualize word embeddings by using output embedding matrix
vectors:  (2854, 768)
query:  (768,)
word: andrew .. neighbors: ['uniform', ')', '##agne', '##fen', 'com', 'loading', 'oils', 'discrete', 'wave', 'different', '(', 'grazing', 'million', 'call', '##at

RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

And here's the code that launches the final evaluation

In [ ]:
# load trained model and evaluate it  
with open(ckpt, 'rb') as f:
    etm_model = torch.load(f)
etm_model = etm_model.to(device)
etm_model.eval()

with torch.no_grad():
    ## ---------------
    ## Idea : get document completion perplexities
    test_ppl = evaluate(etm_model) 

    ## ----------------
    ## Idea : get most used topics
    indices = torch.tensor(range(num_docs_train)) # training documents indices 
    indices = torch.split(indices, batch_size)
    #just initialising data structures 
    thetaAvg = torch.zeros(1, args.num_topics).to(device)
    thetaWeightedAvg = torch.zeros(1, args.num_topics).to(device)
    cnt = 0
    for idx, ind in enumerate(indices):
        data_batch = data.get_batch(ind, args.vocab_size, device) # TODO: fix here 
        sums = data_batch.sum(1).unsqueeze(1) 
        cnt += sums.sum(0).squeeze().cpu().numpy()
        # maybe normalise 
        if args.bow_norm:normalized_data_batch = data_batch / sums
        else: normalized_data_batch = data_batch
        # get the theta 
        theta, _ = model.get_theta(normalized_data_batch)
        thetaAvg += theta.sum(0).unsqueeze(0) / args.num_docs_train
        weighed_theta = sums * theta
        thetaWeightedAvg += weighed_theta.sum(0).unsqueeze(0)
        # let's print the progress as we go 
        if idx % 100 == 0 and idx > 0:
            print('batch: {}/{}'.format(idx, len(indices)))
    # finally the results are in 
    thetaWeightedAvg = thetaWeightedAvg.squeeze().cpu().numpy() / cnt
    print('\nThe 10 most used topics are {}'.format(thetaWeightedAvg.argsort()[::-1][:10]))

    # Now we show the topics
    # A nice visualisation is always welcome 
    beta = model.get_beta()
    topic_indices = list(np.random.choice(num_topics, 10)) # 10 random topics
    print('\n')
    for k in range(num_topics):#topic_indices:
        gamma = beta[k]
        top_words = list(gamma.cpu().numpy().argsort()[-num_words+1:][::-1])
        topic_words = [vocab[a] for a in top_words]
        print('Topic {}: {}'.format(k, topic_words))

    # Why not, also showing a few embeddings 
    if train_embeddings:
        # get embeddings from the model 
        try:rho_etm = model.rho.weight.cpu()
        except:rho_etm = model.rho.cpu()
        queries = ['andrew', 'woman', 'computer', 'sports', 'religion', 'man', 'love', 
                        'intelligence', 'money', 'politics', 'health', 'people', 'family']
        print('\n')
        print('ETM embeddings...')
        for word in queries:
            print('word: {} .. etm neighbors: {}'.format(word, nearest_neighbors(word, rho_etm, vocab)))
        print('\n')
